In [1]:
# This mounts your Google Drive to the Colab VM.
import pandas as pd
import numpy as np
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torchvision.io import read_image
import os
from PIL import Image
from google.colab import drive
import tqdm
import torch.optim as optim
import torch.nn as nn

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
drive.mount('/content/drive')


# TODO: Enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment3/'
# FOLDERNAME = 'class_sampling_chest_xrays/class_sampling_cheXpert_project/'
# assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
FOLDERNAME = 'class_sampling_chest_xrays/class_sampling_cheXpert_project/class-sample-research-main'
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

%cd /content/drive/My Drive/$FOLDERNAME

Mounted at /content/drive
/content/drive/My Drive/class_sampling_chest_xrays/class_sampling_cheXpert_project/class-sample-research-main


In [3]:
import models
import class_sampling
import train
import metric_utils
import inference
import loss_fns
import torchvision.ops
import dataset
import filtered_train_visual_labels

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [4]:
test_labels = pd.read_csv('../../chexlocalize/CheXpert/test_labels.csv')
val_labels = pd.read_csv('../../chexlocalize/CheXpert/val_labels.csv')

In [5]:
train_visual_labels = filtered_train_visual_labels.create_filtered_labels_df()

In [6]:
#TO DO: FILTER OUT LATERAL IMAGES
train_visual_labels = train_visual_labels[train_visual_labels['Frontal/Lateral'].str.contains("Frontal")]

In [7]:
train_visual_labels.head(n=100)

,Path,Sex,Age,Frontal/Lateral,AP/PA,Lung Lesion,Edema,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,No Finding,focal_multifocal_lung_opacity,enlarged_cardiac_silhouette,Condition,has_finding_no_condition,no_finding_has_condition,num_conditions
5,CheXpert-v1.0/train/patient00004/study1/view1_...,Female,20,Frontal,PA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0,False,False,0
11,CheXpert-v1.0/train/patient00006/study1/view1_...,Female,42,Frontal,AP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0,False,False,0
13,CheXpert-v1.0/train/patient00007/study2/view1_...,Male,69,Frontal,AP,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,1,8,False,False,1
16,CheXpert-v1.0/train/patient00009/study1/view1_...,Male,76,Frontal,PA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,8,False,False,1
18,CheXpert-v1.0/train/patient00010/study1/view1_...,Female,50,Frontal,PA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0,False,False,0
20,CheXpert-v1.0/train/patient00011/study13/view1...,Female,22,Frontal,PA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0,False,False,0
40,CheXpert-v1.0/train/patient00013/study1/view1_...,Male,36,Frontal,PA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0,False,False,0
42,CheXpert-v1.0/train/patient00014/study1/view1_...,Female,43,Frontal,PA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0,False,False,0
46,CheXpert-v1.0/train/patient00016/study1/view1_...,Female,54,Frontal,PA,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0,0,6,False,False,1
57,CheXpert-v1.0/train/patient00020/study1/view1_...,Male,63,Frontal,AP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0,False,False,0


In [8]:
base_dir = "/content/drive/My Drive/class_sampling_chest_xrays/chexlocalize/CheXpert/test"

In [9]:
transformations = transforms.Compose(
    [
        transforms.ToTensor(),
        torchvision.transforms.Resize((224,224)),
    ]
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
experiment1_INTER_dataset = class_sampling.Ratio_CheXpert_version(train_visual_labels, 9, [19624, 196, 0, 196, 0, 0, 196, 196, 196], nums=(0,1,2,3,4,5,6,7,8), transform=transformations)

experiment2_INTRA_dataset = class_sampling.Ratio_CheXpert_version(train_visual_labels, 9, [2194, 87, 0, 1759, 0, 0, 87, 87, 87], nums=(0,1,2,3,4,5,6,7,8), transform=transformations)

experiment3_INTER_INTRA_dataset = class_sampling.Ratio_CheXpert_version(train_visual_labels, 9, [87, 87, 0, 1759, 0, 0, 87, 87, 87], nums=(0,1,2,3,4,5,6,7,8), transform=transformations)

experiment4_INTER_INTRA_REVERSED_dataset = class_sampling.Ratio_CheXpert_version(train_visual_labels, 9, [19624, 40, 0, 817, 0, 0, 40, 40, 40], nums=(0,1,2,3,4,5,6,7,8), transform=transformations)

In [ ]:
experiment2_INTRA_dataset = class_sampling.Ratio_CheXpert_version(train_visual_labels, 9, [1983, 82, 0, 82, 0, 0, 1655, 82, 82], nums=(0,1,2,3,4,5,6,7,8), transform=transformations)

experiment2_INTER_INTRA_dataset = class_sampling.Ratio_CheXpert_version(train_visual_labels, 9, [82, 82, 0, 82, 0, 0, 1655, 82, 82], nums=(0,1,2,3,4,5,6,7,8), transform=transformations)

experiment2_INTER_INTRA_REVERSED_dataset = class_sampling.Ratio_CheXpert_version(train_visual_labels, 9, [19624, 40, 0, 40, 0, 0, 817, 40, 40], nums=(0,1,2,3,4,5,6,7,8), transform=transformations)

In [ ]:
experiment3_INTRA_dataset = class_sampling.Ratio_CheXpert_version(train_visual_labels, 9, [1493, 62, 0, 62, 0, 0, 62, 1245, 62], nums=(0,1,2,3,4,5,6,7,8), transform=transformations)

experiment3_INTER_INTRA_dataset = class_sampling.Ratio_CheXpert_version(train_visual_labels, 9, [62, 62, 0, 62, 0, 0, 62, 1245, 62], nums=(0,1,2,3,4,5,6,7,8), transform=transformations)

experiment3_INTER_INTRA_REVERSED_dataset = class_sampling.Ratio_CheXpert_version(train_visual_labels, 9, [19624, 40, 0, 40, 0, 0, 40, 817, 40], nums=(0,1,2,3,4,5,6,7,8), transform=transformations)

In [ ]:
experiment4_INTRA_dataset = class_sampling.Ratio_CheXpert_version(train_visual_labels, 9, [6038, 251, 0, 251, 0, 0, 251, 251, 5034], nums=(0,1,2,3,4,5,6,7,8), transform=transformations)

experiment4_INTER_INTRA_dataset = class_sampling.Ratio_CheXpert_version(train_visual_labels, 9, [251, 251, 0, 251, 0, 0, 251, 251, 5034], nums=(0,1,2,3,4,5,6,7,8), transform=transformations)

experiment4_INTER_INTRA_REVERSED_dataset = class_sampling.Ratio_CheXpert_version(train_visual_labels, 9, [19624, 40, 0, 40, 0, 0, 40, 40, 817], nums=(0,1,2,3,4,5,6,7,8), transform=transformations)

In [ ]:
pick3vsall1_INTRA_dataset = class_sampling.Ratio_CheXpert_version(train_visual_labels, 9, [3858, 62, 0, 62, 0, 0, 1245, 1245, 1245], nums=(0,1,2,3,4,5,6,7,8), transform=transformations)

pick3vsall1_INTER_INTRA_dataset = class_sampling.Ratio_CheXpert_version(train_visual_labels, 9, [62, 62, 0, 62, 0, 0, 1245, 1245, 1245], nums=(0,1,2,3,4,5,6,7,8), transform=transformations)

pick3vsall1_INTER_INTRA_REVERSED_dataset = class_sampling.Ratio_CheXpert_version(train_visual_labels, 9, [19624, 15, 0, 15, 0, 0, 316, 316, 316], nums=(0,1,2,3,4,5,6,7,8), transform=transformations)

In [ ]:
pick3vsall2_INTRA_dataset = class_sampling.Ratio_CheXpert_version(train_visual_labels, 9, [3858, 62, 0, 1245, 0, 0, 62, 1245, 1245], nums=(0,1,2,3,4,5,6,7,8), transform=transformations)

pick3vsall2_INTER_INTRA_dataset = class_sampling.Ratio_CheXpert_version(train_visual_labels, 9, [62, 62, 0, 1245, 0, 0, 62, 1245, 1245], nums=(0,1,2,3,4,5,6,7,8), transform=transformations)

pick3vsall2_INTER_INTRA_REVERSED_dataset = class_sampling.Ratio_CheXpert_version(train_visual_labels, 9, [19624, 15, 0, 316, 0, 0, 15, 316, 316], nums=(0,1,2,3,4,5,6,7,8), transform=transformations)

In [ ]:
pick3vsall3_INTRA_dataset = class_sampling.Ratio_CheXpert_version(train_visual_labels, 9, [5129, 82, 0, 1655, 0, 0, 1655, 82, 1655], nums=(0,1,2,3,4,5,6,7,8), transform=transformations)

pick3vsall3_INTER_INTRA_dataset = class_sampling.Ratio_CheXpert_version(train_visual_labels, 9, [82, 82, 0, 1655, 0, 0, 1655, 82, 1655], nums=(0,1,2,3,4,5,6,7,8), transform=transformations)

pick3vsall3_INTER_INTRA_REVERSED_dataset = class_sampling.Ratio_CheXpert_version(train_visual_labels, 9, [19624, 15, 0, 316, 0, 0, 316, 15, 316], nums=(0,1,2,3,4,5,6,7,8), transform=transformations)

In [ ]:
pick3vsall4_INTRA_dataset = class_sampling.Ratio_CheXpert_version(train_visual_labels, 9, [3858, 62, 0, 1245, 0, 0, 1245, 1245, 62], nums=(0,1,2,3,4,5,6,7,8), transform=transformations)

pick3vsall4_INTER_INTRA_dataset = class_sampling.Ratio_CheXpert_version(train_visual_labels, 9, [62, 62, 0, 1245, 0, 0, 1245, 1245, 62], nums=(0,1,2,3,4,5,6,7,8), transform=transformations)

pick3vsall4_INTER_INTRA_REVERSED_dataset = class_sampling.Ratio_CheXpert_version(train_visual_labels, 9, [19624, 15, 0, 316, 0, 0, 316, 316, 15], nums=(0,1,2,3,4,5,6,7,8), transform=transformations)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(experiment1_INTER_dataset, batch_size=8, shuffle=True,)

In [ ]:
def get_device():
    if torch.cuda.is_available():
        device = 'cuda:0'
    else:
        device = 'cpu'
    return device

device = get_device()

In [ ]:
num_epochs = 5
for epoch in range(num_epochs):
    for local_batch, local_labels in train_dataloader:
        # Transfer to GPU
        local_batch, local_labels = local_batch.to(device), local_labels.to(device)

In [ ]:
model = torch.hub.load('pytorch/vision', 'resnet50', pretrained=True)